# Semantic Analysis Model
First we need the data for the model, the default corpus is a combination of several datasets. the resulting dataset is made of 500,000 comments, that are labelled as offensive or not offensive. 
Using this dataset we  need to convert into into a machine readable structure    
We can use two representation methods, TF-IDF matrix and a Vector representation  
TF-IDF will be more light weight, but Vector will be a stronger representation  
Either way we will need to preprocess the text data of the combined dataset. This should be saved away for ease of use later as this dataset will not be changing once fully gathered and processed.  

We want to split this dataset into a train and test set  
Using this representation we can feed the train dataset to a libary implemented machine learning model and train it.   
Then we can get a result for it's accurary by testing it with our test set.  
We can also consider k-fold cross validation.  

Several models can be tested to find the one which is most effective at this task. Models of note: K-NN, SVM, Neural Networks, BERT.   
The Nerual Networks and BERT will be computationally expensive, consider running on the computer science clusters.  

An Ensemble method can be created by leveraging the data that includes a target label. By creating a model for each of the given labels we can have a dedicated model for detecting a specific type of hate comment, which the hope is will be more sensetive to these types of comments then the general methods, as suggested by litreture. Then by combining these models in a larger model in an OR fashion or by using some form of model taking the results from it's constituent models as input we can then make the final decision.  This can also allow more transparcncy as to why things have been blocked or rejected, being able to state this comment is suspected to be racist, or has been flagged as sexist etc. Therefore we can inform the user what the moderation has detected so that may make changes to their behaviour.  

We choose to reject or accept, or recommend for human moderation, a comment based on the confidence that the model has with it's classification. that is to say how close to a decision boundary that comment falls. Comments with high confidence can be accepted or rejected straight away. Those that are more uncertain, e.g. has a 54% confidence, can be copyed and sent to a queue for a human moderator to deal with. How these messages are dealt with is up to the developer, be they posted until moderated or hidden until moderated is up to them. This will be a very platform dependent choice.   
Additionally the confidence threshold can be set depending on how much a developer trusts the model, the number of human resources they have to dedicate to moderation, the size of their platform, etc.  

Finally functionality for a developer given dataset should be given so that they can use this to create their own models relatively robustly. It should be sufficent for a CVS file with 3 fields, Comment, Class, Target. Comment being the text in question, the Class being the binary class of if this is offensive or not, and Target being who an offensive comment is targeted at, N/A if target models are not wanted to be used. 


In [4]:
#Preprocess our 23 datasets here to strip them down to Comment, Class, Target

#Downloads the berkley dataset to be cleaned.
import datasets
#import pickle
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary') 
df = dataset['train'].to_pandas()

'''
#TODO
#I'll leave this for now as it's only needed for the ensemble methods.
#Coallate the targets of a comment into a single array
for i in range(len(df)):
    #TODO investigate this threshold for True or falsing this
    if df.loc[i,"hate_speech_score"] <= 0:
        df.loc[i,"hate_speech_score"] = 0
    else:
        df.loc[i,"hate_speech_score"] = 1
'''


#Convert the hate_speech_score into a true or a false
for i in range(len(df)):
    #TODO investigate this threshold for True or falsing this
    if df.loc[i,"hate_speech_score"] <= 0:
        df.loc[i,"hate_speech_score"] = 'Ok'
    else:
        df.loc[i,"hate_speech_score"] = 'Hate'
# our attributes are df['text'] df['hate_speech_score'] Turned into a class, and df['targets'] saved into an array of targets, rather than seperate attributes
#df_to_save=df[['text','hate_speech_score']]
#df_to_save.to_pickle("/berkely_dataset.pkl")

Using custom data configuration ucberkeley-dlab--measuring-hate-speech-7cb9b0b8e4d0e1dd
Reusing dataset parquet (/dcs/18/u1801796/.cache/huggingface/datasets/parquet/ucberkeley-dlab--measuring-hate-speech-7cb9b0b8e4d0e1dd/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)
100%|██████████| 1/1 [00:00<00:00, 127.53it/s]


In [9]:
#Take in our combined dataset, in format, Comment, Class, Target and focus on the Comment attribute
#Preprocess comment into a computer readable string representation, either TF-IDF or Vector, should be allowed for either to be used.
# Should be able to deal with any dataset of Comment, Binary Class, Target representation to allow developer to input their own dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from scipy.sparse import hstack
import numpy as np

#TODO combine the class back to the tfidf matrix
def split_dataset(x,y):
    train_x,test_x, train_y,test_y=train_test_split(x, y)
    
    return train_x,test_x,train_y,test_y
def preprocess_train_data(data):
    vectorizer = TfidfVectorizer()
    transformed_data = vectorizer.fit_transform(data) 
    #feature_vector=hstack((transformed_x,y))
    #need to return the vecorizer object to use with the test data
    return transformed_data,vectorizer

def preprocess_test_data(data, vocab):
    vectorizer = TfidfVectorizer(vocabulary=vocab)
    transformed_train = vectorizer.fit_transform(data) 
    #feature_vector=hstack((transformed_x,y))
    return transformed_train
#print(df_to_save[['hate_speech_score']].shape)

def build_and_test(dataset):
    #specific for one data set
    y=dataset['hate_speech_score']
    x=dataset['text']
    train_data,test_data,train_label,test_label=split_dataset(x,y)
    print(train_data.shape, test_data.shape)
    train_data,vectorizer=preprocess_train_data(train_data)
    #build model with train data
    model = KNeighborsClassifier(n_neighbors=21)
    model.fit(train_data, train_label)
    test_data=preprocess_test_data(test_data,vectorizer.vocabulary_)

    print(train_data.shape, test_data.shape)
    #test model against training data
    acc=model.score(test_data,test_label)
    print("Model had accuracy of :".acc)


build_and_test(df)

(101667,) (33889,)


/dcs/18/u1801796/.local/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:1322: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


(101667, 42725) (33889, 42725)


KeyboardInterrupt: 

In [ ]:
# Given a dataset of Preprocessed Text, Binary Class, Target
# Train some models, KNN, vs SVM
# If Target us not N/A for all then we can build a model for each Target, store the models in an array, then we can use these as a feature vector or just OR the results of is confidence>Threshold for all modules.
def train_model(dataset):
    pass

def train_models(dataset):
    pass